In [1]:
import pandas as pd

# 파일 경로에 맞게 파일 읽기
male_df = pd.read_csv("C:/Users/JEONGHEE/Desktop/당뇨병플젝/male_final.csv")
female_df = pd.read_csv("C:/Users/JEONGHEE/Desktop/당뇨병플젝/female_final.csv")

In [3]:
print(male_df.columns) 
print(female_df.columns)

Index(['DS1_ID', 'DS1_SEX', 'DS1_AGE', 'DS1_HTN', 'DS1_FDM', 'DS1_EXER',
       'DS1_WALK', 'DS1_WALKFQ', 'DS1_WALKT', 'DS1_WAIST', 'DS1_HIP',
       'DS1_PULSE', 'DS1_SBP', 'DS1_DBP', 'DS1_BMI', 'DS1_PBF', 'DS1_BODYFAT',
       'DS1_MUSCLE', 'DS1_VISFAT', 'DS1_WHR', 'DS1_GLU0', 'DS1_HBA1C',
       'target', 'DS1_INCOME_RE', 'DS1_EDU_RE', 'DS1_MARRY_RE', 'total_exer',
       'DS1_SMOKE_RE', 'DS1_DRINK_RE', 'BMI_category', 'exer_category',
       'total_walk', 'walk_category'],
      dtype='object')
Index(['DS1_ID', 'DS1_SEX', 'DS1_AGE', 'DS1_HTN', 'DS1_FDM', 'DS1_EXER',
       'DS1_WALK', 'DS1_WALKFQ', 'DS1_WALKT', 'DS1_WAIST', 'DS1_HIP',
       'DS1_PULSE', 'DS1_SBP', 'DS1_DBP', 'DS1_BMI', 'DS1_PBF', 'DS1_BODYFAT',
       'DS1_MUSCLE', 'DS1_VISFAT', 'DS1_WHR', 'DS1_GLU0', 'DS1_HBA1C',
       'target', 'DS1_INCOME_RE', 'DS1_EDU_RE', 'DS1_MARRY_RE', 'total_exer',
       'DS1_SMOKE_RE', 'DS1_DRINK_RE', 'BMI_category', 'exer_category',
       'total_walk', 'walk_category'],
      dtype='o

In [5]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu, chi2_contingency

# 연속형, 범주형 변수 지정
continuous_vars = ['DS1_AGE', 'DS1_HIP','DS1_PULSE', 'DS1_SBP', 'DS1_DBP',
                   'DS1_PBF', 'DS1_MUSCLE', 'DS1_VISFAT', 'DS1_WHR','DS1_BMI']
categorical_vars = ['DS1_INCOME_RE', 'DS1_EDU_RE', 'DS1_MARRY_RE', 'DS1_SMOKE_RE',
                    'DS1_DRINK_RE', 'walk_category', 'exer_category', 'DS1_HTN', 'DS1_FDM']
target = 'target'

def make_summary_table(df, cont_vars, cat_vars, group_col='target'):
    rows = []
    
    for var in cont_vars:
        group0 = df[df[group_col] == 0][var].dropna()
        group1 = df[df[group_col] == 1][var].dropna()
        
        mean_std_0 = f"{group0.mean():.1f} ({group0.std():.1f})"
        mean_std_1 = f"{group1.mean():.1f} ({group1.std():.1f})"
        
        # Mann-Whitney U test 적용
        try:
            _, p = mannwhitneyu(group0, group1, alternative='two-sided')
        except:
            p = np.nan
        
        rows.append([var, mean_std_0, mean_std_1, round(p, 4)])
    
    for var in cat_vars:
        ct = pd.crosstab(df[var], df[group_col])
        try:
            _, p, _, _ = chi2_contingency(ct)
        except:
            p = np.nan
        
        rows.append([var + ' (categorical)', '', '', round(p, 4)])
        
        for level in ct.index:
            n0 = ct.loc[level][0] if 0 in ct.columns else 0
            n1 = ct.loc[level][1] if 1 in ct.columns else 0
            total = n0 + n1
            pct_0 = f"{n0} ({n0 / total * 100:.1f}%)" if total else "0 (0.0%)"
            pct_1 = f"{n1} ({n1 / total * 100:.1f}%)" if total else "0 (0.0%)"
            rows.append([f"  └ {level}", pct_0, pct_1, ''])
    
    return pd.DataFrame(rows, columns=['변수명', '비당뇨', '당뇨', 'p-value'])

# 실행 예시
male_table = make_summary_table(male_df, continuous_vars, categorical_vars)

male_table

,변수명,비당뇨,당뇨,p-value
0,DS1_AGE,53.1 (8.7),54.7 (8.2),0.0
1,DS1_HIP,96.2 (5.5),97.3 (6.1),0.0
2,DS1_PULSE,68.3 (9.6),71.6 (10.5),0.0
3,DS1_SBP,125.5 (14.3),131.0 (14.9),0.0
4,DS1_DBP,78.8 (9.7),81.2 (9.7),0.0
5,DS1_PBF,23.5 (4.2),25.4 (4.1),0.0
6,DS1_MUSCLE,49.1 (5.1),49.7 (5.5),0.0001
7,DS1_VISFAT,2.5 (1.0),3.0 (1.2),0.0
8,DS1_WHR,0.9 (0.1),0.9 (0.0),0.0
9,DS1_BMI,24.4 (2.6),25.5 (2.8),0.0


In [10]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu, chi2_contingency

# 1. 파일 불러오기
male_df = pd.read_csv("C:/Users/JEONGHEE/Desktop/당뇨병플젝/male_final.csv")
female_df = pd.read_csv("C:/Users/JEONGHEE/Desktop/당뇨병플젝/female_final.csv")

# 2. 변수 지정
continuous_vars = ['DS1_AGE', 'DS1_HIP','DS1_PULSE', 'DS1_SBP', 'DS1_DBP',
                   'DS1_PBF', 'DS1_MUSCLE', 'DS1_VISFAT', 'DS1_WHR','DS1_BMI']
categorical_vars = ['DS1_INCOME_RE', 'DS1_EDU_RE', 'DS1_MARRY_RE', 'DS1_SMOKE_RE',
                    'DS1_DRINK_RE', 'walk_category', 'exer_category', 'DS1_HTN', 'DS1_FDM']

# 3. 요약표 함수 정의
def make_summary_table(df, cont_vars, cat_vars, group_col='target'):
    rows = []

    # 연속형 변수: 평균 ± SD, 중앙값 [IQR], Mann-Whitney U Test
    for var in cont_vars:
        group0 = df[df[group_col] == 0][var].dropna()
        group1 = df[df[group_col] == 1][var].dropna()

        # 평균 ± 표준편차
        mean_0 = group0.mean()
        std_0 = group0.std()
        mean_std_0 = f"{mean_0:.1f} ± {std_0:.1f}"

        mean_1 = group1.mean()
        std_1 = group1.std()
        mean_std_1 = f"{mean_1:.1f} ± {std_1:.1f}"

        # 중앙값 [IQR]
        median_0 = group0.median()
        q1_0 = group0.quantile(0.25)
        q3_0 = group0.quantile(0.75)
        iqr_0 = f"{median_0:.1f} [{q1_0:.1f} ~ {q3_0:.1f}]"

        median_1 = group1.median()
        q1_1 = group1.quantile(0.25)
        q3_1 = group1.quantile(0.75)
        iqr_1 = f"{median_1:.1f} [{q1_1:.1f} ~ {q3_1:.1f}]"

        # Mann-Whitney U test
        try:
            _, p = mannwhitneyu(group0, group1, alternative='two-sided')
        except:
            p = np.nan

        # 합치기
        result_0 = f"{mean_std_0} / {iqr_0}"
        result_1 = f"{mean_std_1} / {iqr_1}"

        rows.append([var, result_0, result_1, round(p, 4)])

    # 범주형 변수: 비율 + chi-squared test
    for var in cat_vars:
        ct = pd.crosstab(df[var], df[group_col])
        try:
            _, p, _, _ = chi2_contingency(ct)
        except:
            p = np.nan

        rows.append([var + ' (categorical)', '', '', round(p, 4)])

        col0_total = ct[0].sum() if 0 in ct.columns else 0
        col1_total = ct[1].sum() if 1 in ct.columns else 0

        for level in ct.index:
            n0 = ct.loc[level][0] if 0 in ct.columns else 0
            n1 = ct.loc[level][1] if 1 in ct.columns else 0
            pct_0 = f"{n0} ({n0 / col0_total * 100:.1f}%)" if col0_total else "0 (0.0%)"
            pct_1 = f"{n1} ({n1 / col1_total * 100:.1f}%)" if col1_total else "0 (0.0%)"
            rows.append([f"  └ {level}", pct_0, pct_1, ''])

    return pd.DataFrame(rows, columns=['변수명', '비당뇨 (mean±SD / median[IQR])', '당뇨 (mean±SD / median[IQR])', 'p-value'])

# 4. 남성 요약표 생성
male_table = make_summary_table(male_df, continuous_vars, categorical_vars)

# 5. 여성 요약표 생성
female_table = make_summary_table(female_df, continuous_vars, categorical_vars)

# 6. 출력
print("남성 요약표:")
display(male_table)

남성 요약표:


,변수명,비당뇨 (mean±SD / median[IQR]),당뇨 (mean±SD / median[IQR]),p-value
0,DS1_AGE,53.1 ± 8.7 / 53.0 [46.0 ~ 60.0],54.7 ± 8.2 / 55.0 [48.0 ~ 61.0],0.0
1,DS1_HIP,96.2 ± 5.5 / 96.0 [92.5 ~ 99.7],97.3 ± 6.1 / 97.0 [93.5 ~ 101.0],0.0
2,DS1_PULSE,68.3 ± 9.6 / 68.0 [62.0 ~ 74.0],71.6 ± 10.5 / 71.0 [64.0 ~ 78.0],0.0
3,DS1_SBP,125.5 ± 14.3 / 125.0 [116.0 ~ 134.0],131.0 ± 14.9 / 130.0 [120.0 ~ 140.0],0.0
4,DS1_DBP,78.8 ± 9.7 / 80.0 [71.5 ~ 85.0],81.2 ± 9.7 / 80.0 [75.0 ~ 87.0],0.0
5,DS1_PBF,23.5 ± 4.2 / 23.7 [20.8 ~ 26.4],25.4 ± 4.1 / 25.6 [23.0 ~ 28.0],0.0
6,DS1_MUSCLE,49.1 ± 5.1 / 48.7 [45.7 ~ 52.2],49.7 ± 5.5 / 49.3 [46.0 ~ 52.9],0.0001
7,DS1_VISFAT,2.5 ± 1.0 / 2.4 [1.9 ~ 3.0],3.0 ± 1.2 / 2.8 [2.2 ~ 3.5],0.0
8,DS1_WHR,0.9 ± 0.1 / 0.9 [0.9 ~ 0.9],0.9 ± 0.0 / 0.9 [0.9 ~ 0.9],0.0
9,DS1_BMI,24.4 ± 2.6 / 24.3 [22.6 ~ 26.1],25.5 ± 2.8 / 25.3 [23.7 ~ 27.2],0.0


In [12]:
print("\n여성 요약표:")
display(female_table)


여성 요약표:


,변수명,비당뇨 (mean±SD / median[IQR]),당뇨 (mean±SD / median[IQR]),p-value
0,DS1_AGE,52.0 ± 7.9 / 52.0 [46.0 ~ 58.0],55.9 ± 7.7 / 56.0 [50.0 ~ 62.0],0.0
1,DS1_HIP,93.7 ± 5.5 / 93.5 [90.0 ~ 97.0],95.8 ± 6.6 / 95.0 [91.2 ~ 99.8],0.0
2,DS1_PULSE,69.2 ± 9.2 / 68.0 [63.0 ~ 74.0],72.1 ± 9.8 / 71.0 [65.0 ~ 78.0],0.0
3,DS1_SBP,120.5 ± 15.1 / 120.0 [110.0 ~ 130.0],128.6 ± 16.3 / 128.0 [119.0 ~ 138.0],0.0
4,DS1_DBP,74.6 ± 9.7 / 74.0 [69.0 ~ 80.0],78.6 ± 9.8 / 80.0 [71.0 ~ 84.5],0.0
5,DS1_PBF,30.7 ± 4.1 / 30.8 [28.0 ~ 33.5],33.6 ± 3.9 / 33.7 [31.1 ~ 36.2],0.0
6,DS1_MUSCLE,36.6 ± 3.4 / 36.4 [34.3 ~ 38.7],37.6 ± 4.0 / 37.3 [34.8 ~ 40.1],0.0
7,DS1_VISFAT,2.1 ± 0.9 / 2.0 [1.5 ~ 2.5],2.7 ± 1.0 / 2.6 [2.0 ~ 3.2],0.0
8,DS1_WHR,0.8 ± 0.1 / 0.8 [0.8 ~ 0.9],0.9 ± 0.1 / 0.9 [0.8 ~ 0.9],0.0
9,DS1_BMI,23.7 ± 2.8 / 23.3 [21.6 ~ 25.3],25.8 ± 3.4 / 25.3 [23.3 ~ 27.9],0.0


In [6]:
# 함수 정의: 당뇨 vs 비당뇨 빈도 및 비율 출력
def print_target_distribution(df, label=''):
    count = df['target'].value_counts().sort_index()
    total = count.sum()
    pct = (count / total * 100).round(1)
    
    print(f"{label} 대상자 수 및 비율")
    print(f"- 전체: {total}명")
    for t in count.index:
        label_str = '비당뇨' if t == 0 else '당뇨'
        print(f"- {label_str}: {count[t]}명 ({pct[t]}%)")
    print()

# 실행
print_target_distribution(male_df, '남성')
print_target_distribution(female_df, '여성')


남성 대상자 수 및 비율
- 전체: 36044명
- 비당뇨: 34450명 (95.6%)
- 당뇨: 1594명 (4.4%)

여성 대상자 수 및 비율
- 전체: 68844명
- 비당뇨: 67176명 (97.6%)
- 당뇨: 1668명 (2.4%)

